# Análisis de Ventas de Supermercado

## Objetivo
 - Explorar el comportamiento de las ventas por sucursal, línea de producto, tipo de cliente, género y método de pago.
 - Analizar patrones por fecha y hora.
 - Construir un modelo simple para predecir el monto de compra ('Total') a partir de algunas características del ticket.

In [3]:
# Librerías
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
plt.style.use('ggplot')
sns.set()

## Carga de Datos

In [5]:
df=pd.read_csv('SupermarketSales.csv')
df.head()

,Date,Branch,Customer type,Gender,Product line,Unit price,Quantity,Payment,Rating
0,1/1/2024,Brooklyn,Member,Female,Food & Beverages,84.63,10,Credit card,9.0
1,1/1/2024,Queens,Normal,Female,Electronics,63.22,2,Cash,8.5
2,1/1/2024,Brooklyn,Normal,Female,Electronics,74.71,6,Cash,6.7
3,1/1/2024,Queens,Member,Female,Sports & Travel,36.98,10,Credit card,7.0
4,1/1/2024,Manhattan,Member,Female,Sports & Travel,27.04,4,Ewallet,6.9


In [6]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5053 entries, 0 to 5052
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           5053 non-null   object 
 1   Branch         5053 non-null   object 
 2   Customer type  5053 non-null   object 
 3   Gender         5053 non-null   object 
 4   Product line   5053 non-null   object 
 5   Unit price     5053 non-null   float64
 6   Quantity       5053 non-null   int64  
 7   Payment        5053 non-null   object 
 8   Rating         5053 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 355.4+ KB


,Unit price,Quantity,Rating
count,5053.000000,5053.000000,5053.000000
mean,39.692988,4.593509,6.884029
std,29.557474,2.787934,1.710245
min,2.010000,1.000000,3.000000
25%,11.630000,2.000000,5.600000
50%,33.900000,4.000000,7.100000
75%,64.940000,7.000000,8.200000
max,99.960000,10.000000,10.000000
